Goals
===
1. Scrape data from public Web site: Introduce how to scrape the information of companies from Taiwan Stock Exchange. 如果從公開的資源中抓取資料？：介紹如何從台灣股市交易公司抓取上市公司資料.
      
2. Implement the QueryBoard: make a streamlit artifact which avails for querying company's ticker in stock market and vice vera. 製造一個查詢上市公司的代碼系統.


Note
---
If any error occured, in which displayed warning "xxx Module not found...", use the following to install i:
```
pip install xxx
```

In [1]:
import streamlit as st
import pandas as pd

In [2]:
# https://www.twse.com.tw/zh/page/products/stock-code2.html
# TAI_TWO_ind="https://isin.twse.com.tw/isin/C_public.jsp?strMode=4"

TAI_ind='https://isin.twse.com.tw/isin/C_public.jsp?strMode=2'
# data encoded in Traditional Chinese, 資料使用繁體中文編碼
df=pd.read_html(TAI_ind,encoding='cp950')

In [3]:
# the data in the first page (table)
# only the first feature, 有價證券代號及名稱, we want, 
df[0]

,0,1,2,3,4,5,6
0,有價證券代號及名稱,國際證券辨識號碼(ISIN Code),上市日,市場別,產業別,CFICode,備註
1,股票,股票,股票,股票,股票,股票,股票
2,1101 台泥,TW0001101004,1962/02/09,上市,水泥工業,ESVUFR,NaN
3,1102 亞泥,TW0001102002,1962/06/08,上市,水泥工業,ESVUFR,NaN
4,1103 嘉泥,TW0001103000,1969/11/14,上市,水泥工業,ESVUFR,NaN
...,...,...,...,...,...,...,...
24650,01003T 兆豐新光R1,TW00001003T4,2005/12/26,上市,NaN,CBCIXU,NaN
24651,01004T 土銀富邦R2,TW00001004T2,2006/04/13,上市,NaN,CBCIXU,NaN
24652,01007T 兆豐國泰R2,TW00001007T5,2006/10/13,上市,NaN,CBCIXU,NaN
24653,01009T 王道圓滿R1,TW00001009T1,2018/06/21,上市,NaN,CBCIXU,NaN


In [20]:
# df[0][0][2:]
#   first table, 第一個表格
#      feature 0, 欄位 0
#         start from third rows, i.e. 2

df[0][0][2:].values

array(['1101\u3000台泥', '1102\u3000亞泥', '1103\u3000嘉泥', ...,
       '01007T\u3000兆豐國泰R2', '01009T\u3000王道圓滿R1', '01010T\u3000京城樂富R1'],
      dtype=object)

In [21]:
# extract data and split by '\u3000', 利用字元 '\u3000' 將每一個欄位分成兩個, 
# create two-column DataFrame, 
data = df[0][0][2:].str.split('\u3000', n=1, expand=True)

# create two-column DataFrame, 將上述的兩個公開的資料成為新的欄位
df1 = pd.DataFrame({'Symbol': data[0], 'Name': data[1]})

# convert ticker to yahoo tick,  將上市公司的代碼變成 yahoo 代碼
df1['Symbol'] = df1['Symbol'].apply(lambda x: x + '.TW')

In [22]:
df1.head()

,Symbol,Name
2,1101.TW,台泥
3,1102.TW,亞泥
4,1103.TW,嘉泥
5,1104.TW,環泥
6,1108.TW,幸福


In [24]:
# remove any null value in cell, 去掉沒資料的欄位
df1.fillna('', inplace=True)

# and save to a file, used in later, 並存成檔案供日後使用
df1.to_csv("TWSE_TW-1.csv",index=False)

Techniques by ChatGPT
---
Honestly, this is almost completed by chatGPT, amazing! 這也是 chatGPT 完成的，神奇吧!

In the following Query airtifact is implemented by streamlit; the comments are the statements used for chatGPT, enjoy!. 下列的應用程式是使用 streamlit 完成的，註解裡面的陳述，記載了如何使用 chatGPT.

```python
# 1. If use python to get the data as follows:

#TAI_ind='https://isin.twse.com.tw/isin/C_public.jsp?strMode=2'
#df=pd.read_html(TAI_ind,encoding='cp950')
#df[0][0][2:].values

# 2. Use Python to slice df[0][0][2:] into 2 two column Dataframe by space
# 3. slice by \u3000
# 4. convert each cell in the first column by adding  '.TW'
# 5. use streamlit to make a query system for column1 by column2 and vice versa
# (*)6. 增加中英文 internationalization, (i18n) 選擇

# main code
import streamlit as st
import pandas as pd

# read data from the URL and create DataFrame

df1=pd.read_csv("TWSE_TW-1.csv",index_col=0)
df1.fillna('', inplace=True)
# set up Streamlit app
st.title("TWSE Stock Search, 台灣股票代號查詢")

# add search box and dropdown
search_term = st.text_input("Enter search term, 輸入查詢資料:")
search_by = st.selectbox("Search by column:", options=['公司代碼', '公司名稱'])

# search for matching rows
if search_term:
    if search_by == 'Symbol':
        result = df1[df1['Symbol'].str.contains(search_term)]
    elif search_by == 'Name':
        result = df1[df1['Name'].str.contains(search_term)]
    else:
        result = pd.DataFrame()
    st.write(result)
```

In [27]:
df2=pd.read_csv("TWSE_TW-1.csv")
df2.head()

,Symbol,Name
0,1101.TW,台泥
1,1102.TW,亞泥
2,1103.TW,嘉泥
3,1104.TW,環泥
4,1108.TW,幸福


In [30]:
df1=pd.read_csv("TWSE_TW-1.csv")
df1[df1['Symbol'].str.contains('3008.TW')]

,Symbol,Name
510,3008.TW,大立光
4607,033008.TW,晶華中信26購01
